In [2]:
#Software installs.
# To run the automated linkedin search: install chromedriver for the version of chrome you have:
# https://sites.google.com/a/chromium.org/chromedriver/downloads
!pip install selenium
!pip install bs4

In [12]:
# Import these packages everytime you run this
import pandas as pd 
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs

In [25]:
# Below code is based on the code at the link below (although, somethings are updated due to changes to linkedIn)
# Code source: https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/

#Takes 2 string parameters: email/username, and password
def signin( email, password):

    driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

    # locate email form by_class_name
    username_form = driver.find_element_by_id('username')

    # send_keys() to simulate key strokes
    username_form.send_keys(email)

    time.sleep(0.6)

    # locate password form by_class_name
    password_form = driver.find_element_by_id('password')

    # send_keys() to simulate key strokes
    password_form.send_keys(password)

    time.sleep(0.8)

    # locate submit button by_xpath
    sign_in_button = driver.find_element_by_xpath('//*[@type="submit"]')

    # .click() to mimic button click
    sign_in_button.click()


In [174]:
# **Important** Function for comparing names to the searched name
# Nicknames csv reference: https://github.com/onyxrev/common_nickname_csv
# Note: The nicknames in this file are for "US" names, it will fail
#       on name/nicknames not present (such as Chinese nicknames).
#       Currently, free name/nickname pairing that is more thorough
#       seem to be behind paywalls / not freely available for use.

def nameComparison(first_name, last_name, found_info):
    
    wrong_names = []
    found_names = found_info.keys()
    print(len(found_names))
    
    for found_name in found_names:
        
        first_name_nicknames = nicknames[nicknames["name"] == (" " + first_name)]

        first_name_match = first_name in found_name or found_name in first_name
        
        for nickname in first_name_nicknames["nickname"]:
            nickname_match = nickname.strip() in found_name
            if(nickname_match):
                first_name_match = True
        
        if( first_name_match == False or last_name not in found_name):
            print(found_name + " doesn't match: " + first_name + ' ' + last_name)
            wrong_names.append(found_name)
    
    for wrong_name in wrong_names:
        found_info.pop(wrong_name)
    
    print(len(found_names))

In [179]:
# Testing nameComparison
# Biases:
# Middle initials/names are omitted from matching, as they are often omitted from linkedIn names.
# Different forms of first names are allowed (given that the name exists in the nickname file)
# The only lastname accepted is an exact match with what is on file.

# Issues/TODO: hyphenated lastnames- sometimes only one of the hyphenated names is used => need to detect
# hyphens and allow lastnames split across the hyphen to be acceptable.

first = "Rebecca"
last = "Van Buren"
found_name_dict = {"Becky Van Buren":"asd","Becca Van Buren":"abc",
                   "Reba Van Buren":"idk","Becca Buren":"c","Beck Murillo Van Buren":"hello", 
                   "Rebecca Van Buren":"a","Beck Van":"b"}
nameComparison(first,last,found_name_dict)

7
Becca Buren doesn't match: Rebecca Van Buren
Beck Van doesn't match: Rebecca Van Buren
5


In [178]:
# Function for finding information about each result, original code by myself(Erich)
def resultCompare(first_name, last_name):
    
    #Scraping search results
    results_text = bs(driver.page_source, 'html.parser')
    search_results = results_text.findAll("div",class_="search-result__info pt3 pb4 ph0")
    
    for index in range(0,len(search_results)):

        result = search_results[index]
        found_name_element = result.find("span", class_="name actor-name")
        
        if(found_name_element != None): #LinkedIn members with private visibility show up as None
            found_name = found_name_element.string
            
            #Getting the link to that page
            found_page_link_element = result.find("a")
            found_link = 'https://www.linkedin.com' + found_page_link_element.get('href')
            
            # Adding names and links to a dictionary
            found_info[found_name] = found_link
            print(found_name, found_link)
        
    #uncomment name comparison here when complete
    #nameComparison(first_name, last_name, found_info)
        

In [36]:

# Searching in LinkedIn, original code by myself(Erich)
def linkedIn_search(first_name, last_name):
    #Finding the search bar
    search_bar = driver.find_element_by_class_name('search-global-typeahead__input')
    search_bar.click()

    #Entering in the name/search query
    search_bar.send_keys(first_name + " " + last_name) # The name you want to search
    time.sleep(0.5)

    #Clicking the search button to get the search results
    search_button = driver.find_element_by_class_name('search-global-typeahead__controls')
    search_button.click()

    


In [169]:
#Cleaning nickname csv file:
#Nicknames csv reference: https://github.com/onyxrev/common_nickname_csv

nicknames = pd.read_csv('nicknames.csv')
nicknames = nicknames.drop('id', axis=1)
nicknames.columns = {"name","nickname"}


In [180]:
# Signin & search: Will not compile unless the placeholder fields are replaced

# Chromedriver download: https://sites.google.com/a/chromium.org/chromedriver/downloads

# Replace the placeholder with the absolute path to the chromedriver file 
driver = webdriver.Chrome('absolute_path_to_the_chromedriver')

# Replace the placeholder email and password with your login credentials for automated login to your linkedIn account
signin('youremail','yourpassword')

# The name you want to search goes here ( will be using the last name & first name fields from the alumni file)
first_name = 'Erich'
last_name = 'McMillan'

linkedIn_search(first_name, last_name)

WebDriverException: Message: 'absolute_path_to_the_chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [95]:
# Compare search results to query, determine which profiles to visit
resultCompare(name_to_search)

Erich McMillan https://www.linkedin.com/in/erich-mcmillan-140617185/
Erich McMillan-McCall https://www.linkedin.com/in/erich-mcmillan-mccall-23a16525/
